<a href="https://colab.research.google.com/github/VandadII/colab-notebooks/blob/main/FLUX_1_dev_LoRA_Gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FLUX.1 Gen


In [1]:
!git clone https://github.com/ostris/ai-toolkit
!mkdir -p /content/lora  # this is where your LoRA .safetensors file goes

fatal: destination path 'ai-toolkit' already exists and is not an empty directory.


Put your LoRA in the `/content/lora` folder

In [ ]:
!cd ai-toolkit && git submodule update --init --recursive && pip install -r requirements.txt numpy==1.25.0 --force-reinstall


  Cloning https://github.com/huggingface/diffusers (to revision 363d1ab7e24c5ed6c190abb00df66d9edb74383b) to /tmp/pip-req-build-scz7mz0f
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-scz7mz0f
  Running command git rev-parse -q --verify 'sha^363d1ab7e24c5ed6c190abb00df66d9edb74383b'
  Running command git fetch -q https://github.com/huggingface/diffusers 363d1ab7e24c5ed6c190abb00df66d9edb74383b
  Running command git checkout -q 363d1ab7e24c5ed6c190abb00df66d9edb74383b
  Resolved https://github.com/huggingface/diffusers to commit 363d1ab7e24c5ed6c190abb00df66d9edb74383b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached torchao-0.9.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (14 kB)
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached lycoris_lora-

🔍 Test for ABI mismatches (like numpy dtype size errors)

In [3]:
try:
    import numpy as np
    import diffusers
    from diffusers.schedulers import DDIMScheduler  # or DDPM if that's your config

    print("✅ Environment looks stable. No ABI issues.")
    print("Numpy version:", np.__version__)
except Exception as e:
    print("❌ ABI Error Detected:")
    print(e)

❌ ABI Error Detected:
Failed to import diffusers.schedulers.scheduling_ddim because of the following error (look up to see its traceback):
numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject


## Model License
Training currently only works with FLUX.1-dev. Which means anything you train will inherit the non-commercial license. It is also a gated model, so you need to accept the license on HF before using it. Otherwise, this will fail. Here are the required steps to setup a license.

Sign into HF and accept the model access here [black-forest-labs/FLUX.1-dev](https://huggingface.co/black-forest-labs/FLUX.1-dev)

[Get a READ key from huggingface](https://huggingface.co/settings/tokens/new?) and place it in the next cell after running it.

In [ ]:
import getpass
import os

# Prompt for the token
hf_token = getpass.getpass('Enter your HF access token and press enter: ')

# Set the environment variable
os.environ['HF_TOKEN'] = hf_token

print("HF_TOKEN environment variable has been set.")

In [ ]:
import os
import sys
sys.path.append('/content/ai-toolkit')
from toolkit.job import run_job
from collections import OrderedDict
from PIL import Image
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

## Setup

This is your config. It is documented pretty well. Normally you would do this as a yaml file, but for colab, this will work. This will run as is without modification, but feel free to edit as you want.

In [ ]:
from collections import OrderedDict

job_to_run = OrderedDict([
    ('job', 'generate'),
    ('config', OrderedDict([
        ('name', 'flux_image_gen'),
        ('process', [
            OrderedDict([
                ('type', 'to_folder'),
                ('output_folder', '/content/generated'),  # where your images will be saved
                ('device', 'cuda:0'),
                ('dtype', 'float16'),
                ('model', OrderedDict([
                    ('name_or_path', 'black-forest-labs/FLUX.1-dev'),
                    ('is_flux', True),
                    ('quantize', True),
                    # Optional: Add LoRA file path here to personalize generation.
                    # Leave this line commented if testing the base FLUX.1-dev model only.
                    # ('lora', '/content/lora/vera_000006000.safetensors')  # <-- put your LoRA here
                ])),
                ('generate', OrderedDict([
                    ('sampler', 'flowmatch'),
                    ('width', 1024),
                    ('height', 1024),
                    ('prompts', [
                        'a woman with red hair, playing chess at the park, bomb going off in the background',
                        'a woman holding a coffee cup, in a beanie, sitting at a cafe',
                        'a horse is a DJ at a night club, fish eye lens, smoke machine, laser lights, holding a martini',
                        'a man showing off his cool new t shirt at the beach, a shark is jumping out of the water in the background',
                        'a bear building a log cabin in the snow covered mountains',
                        'woman playing the guitar, on stage, singing a song, laser lights, punk rocker',
                        'hipster man with a beard, building a chair, in a wood shop',
                        'photo of a man, white background, medium shot, modeling clothing, studio lighting, white backdrop',
                        'a man holding a sign that says, "this is a sign"',
                        'a bulldog, in a post apocalyptic world, with a shotgun, in a leather jacket, in a desert, with a motorcycle'
                    ]),
                    ('neg', ''),
                    ('seed', 42),
                    ('guidance_scale', 4),
                    ('sample_steps', 20),
                    ('num_repeats', 1)
                ]))
            ])
        ])
    ]))
])


## Run it

In [ ]:
run_job(job_to_run)
